In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

from tqdm import tqdm
from ft_utils.BatchImageGenerator import BatchImageGenerator
from ft_utils.BatchImageClassifier import BatchImageClassifier
from ft_utils.utils import create_grid_from_batch, BATCH_SIZE, create_image_grid, image_prefix, diff_percentage
import numpy as np
import torch

/Users/felix/.pyenv/versions/3.12.10/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
attributes = ["blonde", "brunette"]

batch_classifier = BatchImageClassifier("out_batch_transfer")
batch_generator = BatchImageGenerator("out_batch_transfer", True)

scores = []
latent_vectors_list = []

text_features = batch_classifier.tokenize_attributes(attributes)

print("Scoring images")
for i in tqdm(range(0, round(200_000 / BATCH_SIZE))):
    probs = batch_classifier.classify_from_batch(i*BATCH_SIZE, BATCH_SIZE, text_features)
    scores.extend([t[0,0].item() for t in probs]) # Use extend for efficiency
    latent_vectors_list.append(batch_generator.load_w_batch(i*BATCH_SIZE, BATCH_SIZE))

latent_vectors = np.concatenate(latent_vectors_list, axis=0)
scores = np.array(scores).reshape(-1, 1)

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/transfer-learning-source-nets/ffhq-res256-mirror-paper256-noaug.pkl"...
Scoring images


100%|██████████| 3125/3125 [00:04<00:00, 671.11it/s]


In [16]:
import ft_utils.InterfaceGANMethod
import ft_utils.AverageMethod
import ft_utils.GaussianMethod

import importlib
importlib.reload(ft_utils.InterfaceGANMethod)
importlib.reload(ft_utils.AverageMethod)
importlib.reload(ft_utils.GaussianMethod)

methods = {
    "InterfaceGAN": ft_utils.InterfaceGANMethod.InterfaceGANMethod(),
    "Average": ft_utils.AverageMethod.AverageMethod(),
    "Gaussian": ft_utils.GaussianMethod.GaussianMethod()
}

print("Starting training...")
for method_name, method in methods.items():
    print("{}: Training...".format(method_name))
    method.train(latent_vectors, scores)

print("Finished training")

Starting training...
InterfaceGAN: Training...
Finish training.
Average: Training...
Gaussian: Training...
Finished training


/Users/felix/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/felix/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/felix/.pyenv/versions/3.12.10/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/felix/Desktop/Privat/Bakk/gan-tut/stylegan2-ada-pytorch/ft_utils/GaussianMethod.py:27: RuntimeWarning: divide by zero encountered in matmul
  self.__cov = (centered.T @ centered) / (latent_vectors_best.shape[0] - 1)
/Users/felix/Desktop/Privat/Bakk/gan-tut/stylegan2-ada-pytorch/ft_utils/GaussianMethod.py:27: RuntimeWarning: overflow encountered in matmul
  self.__cov = (centered.T @ centered) / (latent_vectors_best.shape[0] - 1)
/Users/felix/Desktop/Privat/Bakk/gan-tut/stylegan2-ada-pytorch/ft_utils/Gaussi

In [17]:
print("Measuring...")
batch_idx = 20
seed_idx = 4
first_seed_w_vector = batch_generator.load_w_batch(batch_idx*BATCH_SIZE, BATCH_SIZE)[seed_idx:seed_idx+1]
ws_first = torch.from_numpy(first_seed_w_vector.astype(np.float32)[np.newaxis, :].repeat(14, axis=1)).to("mps")
original_image = batch_generator.generate_from_w_vec(ws_first[0], filename="method_results/comp_original.png")
original_scores = batch_classifier.classify_image_vec(original_image, text_features)

print("Original diff: {:+.2f} %"
    .format(diff_percentage(original_scores[0][0], original_scores[0][1]))
)
print("Original score: {:.2f}"
    .format(original_scores[0][0])
)

for method_name, method in methods.items():
    resulting_w_vector = method.latent_walk(first_seed_w_vector)
    ws = torch.from_numpy(resulting_w_vector.astype(np.float32)[np.newaxis, :].repeat(14, axis=1)).to("mps")
    image = batch_generator.generate_from_w_vec(ws[0], filename=f"method_results/comp_{method_name}.png")
    new_scores = batch_classifier.classify_image_vec(image, text_features)
    print("{} diff: {:+.2f} %".format(method_name, diff_percentage(new_scores[0][0], new_scores[0][1])))
    print("Original score: {:.2f}"
        .format(new_scores[0][0])
    )

Measuring...
Original diff: -11.05 %
Original score: 0.19
InterfaceGAN diff: +11.22 %
Original score: 0.23
Average diff: +1.44 %
Original score: 0.21
Gaussian diff: +0.47 %
Original score: 0.21


/Users/felix/Desktop/Privat/Bakk/gan-tut/stylegan2-ada-pytorch/ft_utils/GaussianMethod.py:47: RuntimeWarning: divide by zero encountered in matmul
  return np.array([self.nearest_ellipsoid_boundary(x)])
/Users/felix/Desktop/Privat/Bakk/gan-tut/stylegan2-ada-pytorch/ft_utils/GaussianMethod.py:47: RuntimeWarning: overflow encountered in matmul
  return np.array([self.nearest_ellipsoid_boundary(x)])
/Users/felix/Desktop/Privat/Bakk/gan-tut/stylegan2-ada-pytorch/ft_utils/GaussianMethod.py:47: RuntimeWarning: invalid value encountered in matmul
  return np.array([self.nearest_ellipsoid_boundary(x)])
